In [1]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [2]:
import sys,os,errno,gc
sys.path.append('bin')
import numpy as np
import argparse, sys, os, errno
from glob import glob
import pandas as pd
import h5py
from tqdm import tqdm
import keras as K
from keras.callbacks import EarlyStopping
from rnnseq2seq import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = '4'
import tensorflow as tf
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects

/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
ic_shape = {}
with h5py.File('known/known.h5') as f:
    for i in tqdm(range(540)):
        ic_shape[i] = f['known'][f['start'][i]:f['end'][i]]
    icshape = np.array([value for (key,value) in ic_shape.iteritems()])
    name = f['name'][:]

100%|██████████| 540/540 [00:00<00:00, 1150.03it/s]


In [14]:
def Model(model_path):
    #prepare for model to predict
    model = seq2seq_model('4',en_depth =4, de_depth =5,dep=4,hid_dim = 10,lr=0.05)
    model.load_weights(model_path)
    return model
model = Model('output/rnnseq2seq_model4_weights.hdf5')

In [6]:
seq1d = {}
with h5py.File('known/1dseq') as f:
    for i in range(540):
        seq1d[i] = f[str(i)][:]

In [33]:
seq1d[0].shape

(358, 128, 4)

In [22]:
icshape[0].shape

(357,)

In [26]:
data =model.predict(seq1d[0])

In [27]:
data.shape

(358, 128, 1)

In [29]:
data[:,:,0].shape

(358, 128)

In [32]:
count =540
save_path = 'output/acc/rnnseq2seq_model4_weights'
def calculate_acc(array,true_score):
    '''
        an array of shape (length)*128
        对齐并且求中间每个位置的平均
        return shape*shape+128
        '''
    shape = array.shape[0]
    new = np.ndarray([shape,shape+128])
    for i in range(shape):
        new[i] = np.concatenate((np.concatenate((np.zeros(i),array[i])),np.zeros(shape-i)))
    score_vector = np.sum(new,axis = 0)[64:-64].astype('float')  #vector  shape
    #这里要分情况！ 长度 64 128为界限  count_vector不一样
    #64以下 每个位置被算长度次
    if shape <=64:
        count_vector = np.repeat(shape,shape)
    if shape >=128:
        count_vector = np.concatenate((np.concatenate((np.arange(65,129),np.repeat(128,shape-128))),~np.arange(65,129) +193))
    if shape >64 and shape <128:
        count_vector = np.concatenate((np.concatenate((np.arange(65,shape+1),np.repeat(shape,128-shape))),~np.arange(65,shape+1) +shape+1 +65))
    score = score_vector/count_vector
    for i in range(shape):
        if score[i] <0.5:
            score[i] = 0
        else:
            score[i] = 1
    acc = float(np.where(np.abs(score-true_score) ==0)[0].shape[0])/float(shape)
    return acc

acc= {}
predict_result = {}
with h5py.File('known/1dseq') as f:
    for i in tqdm(range(count)):
        images = f[str(i)][:]
        predictval = model.predict(images)
        if True:
            with h5py.File(save_path+'wholeprediction.hdf5') as m:
                m.create_dataset(str(i),data = predictval)
        acc[i] = calculate_acc(predictval[:,:,0], icshape[i])
        with h5py.File(save_path) as t:
            t.create_dataset(str(i),data = acc[i])
acc = np.array([val for (key,val) in acc.iteritems()])
np.savetxt(save_path+'_eachseq.txt',acc)

category = np.ndarray([count]).astype('S')
for i in range(count):
    category[i] = name[i].split('_')[0]
name_list = np.unique(category,)
acc_cate = {}
for j in range(10):
    acc_cate[name_list[j]] = []
    for i in range(count):
        if category[i] ==name_list[j]:
            acc_cate[name_list[j]].append(acc[i])
table = pd.DataFrame([10])
for i in range(10):
    table[i] =  sum(acc_cate[name_list[i]])/len(acc_cate[name_list[i]])
table = table.T
table = table.set_index(name_list)
table.columns = ['rnn_model']
#table['2d_model_restrict'] = np.array(pd.read_csv('known/accuracy/acc_1.30_3')['2d_model'])
table['dense'] = ['0.597','0.684','0.628','0.634','0.587','0.524','0.578','0.563','0.555','0.602']
table = table.round(3)

  0%|          | 0/540 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (358,) (357,) 